In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import numpy as np


/content/drive/MyDrive/A_doc_imp/A_Udeyme/A_Python_and_Pytorch_for_generative_AI/Pytorch/data/X.npy

In [ ]:
#device confif
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
#hyperparameters
input_size=4096 # 64*64 images
hidden_size1 = 2048  # First hidden layer
hidden_size2 = 1024  # Second hidden layer
hidden_size3 = 512   # Third hidden layer
hidden_size4 = 256   # Fourth hidden layer
num_classes=10
num_epoch=500
batch_size=50
learning_rate=0.001


In [ ]:
# Load MNIST dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
X=np.load('/content/drive/MyDrive/A_doc_imp/A_Udeyme/A_Python_and_Pytorch_for_generative_AI/Pytorch/data/X.npy')
Y=np.load('/content/drive/MyDrive/A_doc_imp/A_Udeyme/A_Python_and_Pytorch_for_generative_AI/Pytorch/data/Y.npy')


In [ ]:
X.shape

(2062, 64, 64)

In [ ]:
Y.shape

(2062, 10)

In [ ]:
Y[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
#Normalize and resshape the dataset
X=X.reshape(-1,64*64)/255.0

In [ ]:
X.shape


(2062, 4096)

In [ ]:
Y.shape

(2062, 10)

In [ ]:
X=torch.tensor(X,dtype=torch.float32)
Y = np.argmax(Y, axis=1)  # Convert one-hot to class indices
Y=torch.tensor(Y,dtype=torch.long)
Y.shape

torch.Size([2062])

In [ ]:
# Z = np.array([[1, 0, 0],   # Class 0
#               [0, 1, 0],   # Class 1
#               [0, 0, 1]])  # Class 2

# Z = np.argmax(Z, axis=1)  # Converts to class indices
# print(Y)  # Output: [0, 1, 2]

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.15,random_state=42)

In [ ]:
# crearte custom dataset
train_dataset=torch.utils.data.TensorDataset(X_train,Y_train)
test_dataset=torch.utils.data.TensorDataset(X_test,Y_test)

In [ ]:
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=False)


In [ ]:
class ANN(nn.Module):
    def __init__(self):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.fc5 = nn.Linear(hidden_size4, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x),negative_slope=0.01)
        x = F.elu(self.fc3(x))
        x = F.silu(self.fc4(x))
        x = self.fc5(x)  # Output layer
        return x

In [ ]:
model=ANN().to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 20 == 0:
            print(f'Epoch [{epoch+1}/{num_epoch}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/500], Step [20/36], Loss: 2.3141
Epoch [2/500], Step [20/36], Loss: 2.2998
Epoch [3/500], Step [20/36], Loss: 2.3004
Epoch [4/500], Step [20/36], Loss: 2.1871
Epoch [5/500], Step [20/36], Loss: 2.2459
Epoch [6/500], Step [20/36], Loss: 2.0739
Epoch [7/500], Step [20/36], Loss: 2.1676
Epoch [8/500], Step [20/36], Loss: 2.0043
Epoch [9/500], Step [20/36], Loss: 1.9422
Epoch [10/500], Step [20/36], Loss: 2.0155
Epoch [11/500], Step [20/36], Loss: 1.7451
Epoch [12/500], Step [20/36], Loss: 2.0061
Epoch [13/500], Step [20/36], Loss: 1.8309
Epoch [14/500], Step [20/36], Loss: 1.9879
Epoch [15/500], Step [20/36], Loss: 1.7847
Epoch [16/500], Step [20/36], Loss: 1.9252
Epoch [17/500], Step [20/36], Loss: 1.9585
Epoch [18/500], Step [20/36], Loss: 2.1784
Epoch [19/500], Step [20/36], Loss: 1.9816
Epoch [20/500], Step [20/36], Loss: 2.2567
Epoch [21/500], Step [20/36], Loss: 1.9621
Epoch [22/500], Step [20/36], Loss: 1.8863
Epoch [23/500], Step [20/36], Loss: 1.9092
Epoch [24/500], Step

In [ ]:
# testing the data
model.eval()
with torch.no_grad():
  correct=0
  total=0
  for images,labels in test_loader:
    images,labels=images.to(device),labels.to(device)
    outputs=model(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()
print(f'Accuracy on test data{100*correct/total:.2f}')

Accuracy on test data78.71


In [ ]:
# training the data
model.eval()
with torch.no_grad():
  correct=0
  total=0
  for images,labels in train_loader:
    images,labels=images.to(device),labels.to(device)
    outputs=model(images)
    _,predicted=torch.max(outputs.data,1)
    total+=labels.size(0)
    correct+=(predicted==labels).sum().item()
print(f'Accuracy on train data{100*correct/total:.2f}')

Accuracy on train data94.52


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# outputs.data = torch.tensor([
#     [2.5, 1.2, 0.8, 5.0],  # Image 1
#     [3.0, 2.5, 1.2, 1.1]   # Image 2
# ])

In [ ]:
# _, predicted = torch.max(outputs.data, 1)


In [ ]:
# predicted = torch.tensor([9, 0])
# This means that the model predicted class 9 for the first example and class 0 for the second example